To establish communication between Visual Odometry and Kalman filter, homogeneous transformation matrix is needed.
Currently, filters that adopt velocity motion model contains position and angle in x and z-axis to estimate the motion of an agent in a system.

To compute homogeneous transformation matrix, translation vector that contains position in x, y, and z coordinates, and rotation matrix, which can be derived from quaternion, are needed.
In this notebook, mathematical equation that explains new kinematic equation, which conforms to the velocity motion model, is derived.

When converting euler angles from quaternion, gimbal lock, which occurs when pitch angle (angle in y-axis) is +90 or -90 degree, needs to be taken into account.

In [5]:
import numpy as np
import sympy
from sympy.abc import alpha, x, y, v, w, R, psi, theta, phi
from sympy.stats import Normal
from sympy import symbols, Matrix

In [8]:
# state vector

# position
px = symbols('p_{x}')
py = symbols('p_{y}')
pz = symbols('p_{z}')

# velocity
vx = symbols('v_{x}')
vy = symbols('v_{y}')
vz = symbols('v_{z}')

# quaternion
qw = symbols('q_{w}')
qx = symbols('q_{x}')
qy = symbols('q_{y}')
qz = symbols('q_{z}')

# bias for angular velocity
bwx = symbols('b_{w_x}')
bwy = symbols('b_{w_y}')
bwz = symbols('b_{w_z}')

# bias for linear acceleration
bax = symbols('b_{a_x}')
bay = symbols('b_{a_y}')
baz = symbols('b_{a_z}')

# control input
vf = symbols('v_{f}')

ax = symbols('a_{x}')
ay = symbols('a_{y}')
az = symbols('a_{z}')

# Linear acceleration
a = Matrix([[ax], [ay], [az]])

wx = symbols('w_{x}')
wy = symbols('w_{y}')
wz = symbols('w_{z}')

sigma_w = symbols('sigma_{w}')
sigma_a = symbols('sigma_{a}')

# angular velocity
w = Matrix([[wx], [wy], [wz]])

# delta time
dt = symbols('dt')

# gravitational acceleration
g = Matrix([[0], [0], [-9.81]])

In [13]:
norm_w = symbols('|w|')
R = Matrix([
    [(qw**2 + qx**2 - qy**2 - qz**2), 2*(qx*qy - qw*qz), 2*(qx*qz + qw*qy)],
    [2*(qx*qy + qw*qz), qw**2 - qx**2 + qy**2 - qz**2, 2*(qy*qz - qw*qx)],
    [2*(qx*qz - qw*qy), 2*(qy*qz + qw*qx), qw**2 - qx**2 - qy**2 + qz**2]
])

R_T = R.transpose()

Omega = Matrix([
    [0, -wx, -wy, -wz],
    [wx, 0, wz, -wy],
    [wy, -wz, 0, wx],
    [wz, wy, -wx, 0]
])

In [10]:
p = Matrix([[px], [py], [pz]])
v = Matrix([[vx], [vy], [vz]])
q = Matrix([[qw], [qx], [qy], [qz]])
bw = Matrix([[bwx], [bwy], [bwz]])
ba = Matrix([[bax], [bay], [baz]])

ww = Normal("w_{w}", 0, sigma_w)
wa = Normal("w_{a}", 0, sigma_a)

state_x = Matrix([px, py, pz, vx, vy, vz, qw, qx, qy, qz, bwx, bwy, bwz, bax, bay, baz])
control_input = Matrix([ax, ay, az, wx, wy, wz])

In [11]:
a_world = (R @ a + g)

p_t = p + v * dt + 1/2 * a_world * dt**2
v_t = v + a_world * dt

A = sympy.cos(norm_w*dt/2) * sympy.eye(4)
B = (1/norm_w)*sympy.sin(norm_w*dt/2)
q_t = (A + B * Omega) * q

b_w_t = Matrix([
    [bwx + ww],
    [bwy + ww],
    [bwz + ww]
])
b_a_t = Matrix([
    [bax + wa],
    [bay + wa],
    [baz + wa]
])

## Position update

In [20]:
z_position = Matrix([[0], [0], [0]])

hx_position = Matrix([
    [px],
    [py],
    [pz]
])

H_jacobian_position = hx_position.jacobian(state_x)

In [21]:
H_jacobian_position

Matrix([
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

## Velocity update

In [23]:
z_velocity = Matrix([[0], [0], [0]])

v_z = R_T @ v

hx_velocity = Matrix([
    v_z
])

H_jacobian_velocity = hx_velocity.jacobian(state_x)

In [41]:
hx_velocity

Matrix([
[v_{x}*(q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2) + v_{y}*(2*q_{w}*q_{z} + 2*q_{x}*q_{y}) + v_{z}*(-2*q_{w}*q_{y} + 2*q_{x}*q_{z})],
[v_{x}*(-2*q_{w}*q_{z} + 2*q_{x}*q_{y}) + v_{y}*(q_{w}**2 - q_{x}**2 + q_{y}**2 - q_{z}**2) + v_{z}*(2*q_{w}*q_{x} + 2*q_{y}*q_{z})],
[v_{x}*(2*q_{w}*q_{y} + 2*q_{x}*q_{z}) + v_{y}*(-2*q_{w}*q_{x} + 2*q_{y}*q_{z}) + v_{z}*(q_{w}**2 - q_{x}**2 - q_{y}**2 + q_{z}**2)]])

In [24]:
H_jacobian_velocity

Matrix([
[0, 0, 0, q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2,             2*q_{w}*q_{z} + 2*q_{x}*q_{y},            -2*q_{w}*q_{y} + 2*q_{x}*q_{z}, 2*q_{w}*v_{x} - 2*q_{y}*v_{z} + 2*q_{z}*v_{y},  2*q_{x}*v_{x} + 2*q_{y}*v_{y} + 2*q_{z}*v_{z}, -2*q_{w}*v_{z} + 2*q_{x}*v_{y} - 2*q_{y}*v_{x},  2*q_{w}*v_{y} + 2*q_{x}*v_{z} - 2*q_{z}*v_{x}, 0, 0, 0, 0, 0, 0],
[0, 0, 0,            -2*q_{w}*q_{z} + 2*q_{x}*q_{y}, q_{w}**2 - q_{x}**2 + q_{y}**2 - q_{z}**2,             2*q_{w}*q_{x} + 2*q_{y}*q_{z}, 2*q_{w}*v_{y} + 2*q_{x}*v_{z} - 2*q_{z}*v_{x},  2*q_{w}*v_{z} - 2*q_{x}*v_{y} + 2*q_{y}*v_{x},  2*q_{x}*v_{x} + 2*q_{y}*v_{y} + 2*q_{z}*v_{z}, -2*q_{w}*v_{x} + 2*q_{y}*v_{z} - 2*q_{z}*v_{y}, 0, 0, 0, 0, 0, 0],
[0, 0, 0,             2*q_{w}*q_{y} + 2*q_{x}*q_{z},            -2*q_{w}*q_{x} + 2*q_{y}*q_{z}, q_{w}**2 - q_{x}**2 - q_{y}**2 + q_{z}**2, 2*q_{w}*v_{z} - 2*q_{x}*v_{y} + 2*q_{y}*v_{x}, -2*q_{w}*v_{y} - 2*q_{x}*v_{z} + 2*q_{z}*v_{x},  2*q_{w}*v_{x} - 2*q_{y}*v_{z} + 2*q_{z}*v_{y},  2*q_{x}*v_

## Lateral and Upward velocity update

In [62]:
z_velocity = Matrix([[0], [0]])

v_z = R_T @ v

hx_velocity = Matrix([
    v_z[:2]
])

H_jacobian_velocity = hx_velocity.jacobian(state_x)

In [63]:
H_jacobian_velocity

Matrix([
[0, 0, 0, q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2,             2*q_{w}*q_{z} + 2*q_{x}*q_{y}, -2*q_{w}*q_{y} + 2*q_{x}*q_{z}, 2*q_{w}*v_{x} - 2*q_{y}*v_{z} + 2*q_{z}*v_{y}, 2*q_{x}*v_{x} + 2*q_{y}*v_{y} + 2*q_{z}*v_{z}, -2*q_{w}*v_{z} + 2*q_{x}*v_{y} - 2*q_{y}*v_{x},  2*q_{w}*v_{y} + 2*q_{x}*v_{z} - 2*q_{z}*v_{x}, 0, 0, 0, 0, 0, 0],
[0, 0, 0,            -2*q_{w}*q_{z} + 2*q_{x}*q_{y}, q_{w}**2 - q_{x}**2 + q_{y}**2 - q_{z}**2,  2*q_{w}*q_{x} + 2*q_{y}*q_{z}, 2*q_{w}*v_{y} + 2*q_{x}*v_{z} - 2*q_{z}*v_{x}, 2*q_{w}*v_{z} - 2*q_{x}*v_{y} + 2*q_{y}*v_{x},  2*q_{x}*v_{x} + 2*q_{y}*v_{y} + 2*q_{z}*v_{z}, -2*q_{w}*v_{x} + 2*q_{y}*v_{z} - 2*q_{z}*v_{y}, 0, 0, 0, 0, 0, 0]])

## Quaternion update

In [27]:
z_quaternion = Matrix([[0], [0], [0], [0]])

hx_quaternion = Matrix([
    q
])

H_jacobian_quaternion = hx_quaternion.jacobian(state_x)

In [28]:
H_jacobian_quaternion

Matrix([
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]])